In [16]:
import pandas as pd
import numpy as np
from sklearn import model_selection
from sklearn import metrics
import seaborn as sns

from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

In [17]:
df = pd.read_csv("odb.csv")

In [18]:
df.head()

,Unnamed: 0,Player,Span,Mat,Inns,NO,Runs,HS,Ave,BF,SR,100,50,0,4s,6s
0,0,SR Tendulkar (INDIA),1989-2012,463,452,41,18426,200*,44.83,21368,86.23,49,96,20,2016,195
1,1,V Kohli (INDIA),2008-2021,254,245,39,12169,183,59.07,13061,93.17,43,62,13,1140,125
2,2,RT Ponting (AUS/ICC),1995-2012,375,365,39,13704,164,42.03,17046,80.39,30,82,20,1231,162
3,3,RG Sharma (INDIA),2007-2021,227,220,32,9205,264,48.96,10354,88.90,29,43,13,832,244
4,4,ST Jayasuriya (Asia/SL),1989-2011,445,433,18,13430,189,32.36,14725,91.20,28,68,34,1500,270


In [5]:
df.tail()

,Unnamed: 0,Player,Span,Mat,Inns,NO,Runs,HS,Ave,BF,SR,100,50,0,4s,6s
114,114,SK Raina (INDIA),2005-2018,226,194,35,5615,116*,35.31,6005,93.50,5,36,14,476,120
115,115,H Masakadza (ZIM),2001-2019,209,208,4,5658,178*,27.73,7728,73.21,5,34,15,585,86
116,116,RB Richardson (WI),1983-1996,224,217,30,6248,122,33.41,9801,63.74,5,44,8,541+,54+
117,117,BB McCullum (NZ),2002-2016,260,228,28,6083,166,30.41,6312,96.37,5,32,20,577,200
118,118,Saleem Malik (PAK),1982-1999,283,256,38,7170,102,32.88,9383,76.41,5,47,19,514+,34+


In [6]:
df.var()

C:\Users\ajeet\AppData\Local\Temp\ipykernel_12364\1568254755.py:1: FutureWarning: The default value of numeric_only in DataFrame.var is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.var()


Unnamed: 0    1.190000e+03
Mat           9.002154e+03
Inns          7.941532e+03
NO            2.502101e+02
Runs          9.555442e+06
Ave           4.918985e+01
BF            1.516984e+07
SR            1.300275e+02
100           5.742145e+01
50            4.002118e+02
0             3.994203e+01
dtype: float64

In [19]:
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df


,Player,Span,Mat,Inns,NO,Runs,HS,Ave,BF,SR,100,50,0,4s,6s
0,SR Tendulkar (INDIA),1989-2012,463,452,41,18426,200*,44.83,21368,86.23,49,96,20,2016,195
1,V Kohli (INDIA),2008-2021,254,245,39,12169,183,59.07,13061,93.17,43,62,13,1140,125
2,RT Ponting (AUS/ICC),1995-2012,375,365,39,13704,164,42.03,17046,80.39,30,82,20,1231,162
3,RG Sharma (INDIA),2007-2021,227,220,32,9205,264,48.96,10354,88.90,29,43,13,832,244
4,ST Jayasuriya (Asia/SL),1989-2011,445,433,18,13430,189,32.36,14725,91.20,28,68,34,1500,270
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,SK Raina (INDIA),2005-2018,226,194,35,5615,116*,35.31,6005,93.50,5,36,14,476,120
115,H Masakadza (ZIM),2001-2019,209,208,4,5658,178*,27.73,7728,73.21,5,34,15,585,86
116,RB Richardson (WI),1983-1996,224,217,30,6248,122,33.41,9801,63.74,5,44,8,541+,54+
117,BB McCullum (NZ),2002-2016,260,228,28,6083,166,30.41,6312,96.37,5,32,20,577,200


In [20]:
df.dtypes


Player     object
Span       object
Mat         int64
Inns        int64
NO          int64
Runs        int64
HS         object
Ave       float64
BF          int64
SR        float64
100         int64
50          int64
0           int64
4s         object
6s         object
dtype: object

In [23]:
df['HS'] = df['HS'].str.replace('*', '').astype(int)
df['4s'] = df['4s'].str.replace('+', '').astype(int)
df['6s'] = df['6s'].str.replace('+', '').astype(int)


In [24]:
df.head()

,Player,Span,Mat,Inns,NO,Runs,HS,Ave,BF,SR,100,50,0,4s,6s
0,SR Tendulkar (INDIA),1989-2012,463,452,41,18426,200,44.83,21368,86.23,49,96,20,2016,195
1,V Kohli (INDIA),2008-2021,254,245,39,12169,183,59.07,13061,93.17,43,62,13,1140,125
2,RT Ponting (AUS/ICC),1995-2012,375,365,39,13704,164,42.03,17046,80.39,30,82,20,1231,162
3,RG Sharma (INDIA),2007-2021,227,220,32,9205,264,48.96,10354,88.90,29,43,13,832,244
4,ST Jayasuriya (Asia/SL),1989-2011,445,433,18,13430,189,32.36,14725,91.20,28,68,34,1500,270


In [25]:
df.dtypes

Player     object
Span       object
Mat         int64
Inns        int64
NO          int64
Runs        int64
HS          int32
Ave       float64
BF          int64
SR        float64
100         int64
50          int64
0           int64
4s          int32
6s          int32
dtype: object

In [26]:
df['Ave'] = df['Ave'].round().astype(int)
df['SR'] = df['SR'].round().astype(int)

In [31]:
unique_players = df['Player'].unique()
print(unique_players)

['SR Tendulkar (INDIA)' 'V Kohli (INDIA)' 'RT Ponting (AUS/ICC)'
 'RG Sharma (INDIA)' 'ST Jayasuriya (Asia/SL)' 'HM Amla (SA)'
 'AB de Villiers (Afr/SA)' 'CH Gayle (ICC/WI)'
 'KC Sangakkara (Asia/ICC/SL)' 'SC Ganguly (Asia/INDIA)' 'TM Dilshan (SL)'
 'LRPL Taylor (NZ)' 'HH Gibbs (SA)' 'Saeed Anwar (PAK)' 'BC Lara (ICC/WI)'
 'DPMD Jayawardene (Asia/SL)' 'DA Warner (AUS)' 'ME Waugh (AUS)'
 'AJ Finch (AUS)' 'S Dhawan (INDIA)' 'DL Haynes (WI)'
 'JH Kallis (Afr/ICC/SA)' 'Q de Kock (SA)' 'JE Root (ENG)'
 'MJ Guptill (NZ)' 'NJ Astle (NZ)' 'AC Gilchrist (AUS/ICC)'
 'WU Tharanga (Asia/SL)' 'V Sehwag (Asia/ICC/INDIA)'
 'Mohammad Yousuf (Asia/PAK)' 'Babar Azam (PAK)' 'Tamim Iqbal (BAN)'
 'EJG Morgan (ENG/IRE)' 'Yuvraj Singh (Asia/INDIA)' 'KS Williamson (NZ)'
 'G Kirsten (SA)' 'ME Trescothick (ENG)' 'PR Stirling (IRE)'
 'F du Plessis (SA)' 'R Dravid (Asia/ICC/INDIA)' 'JM Bairstow (ENG)'
 'SPD Smith (AUS)' 'CG Greenidge (WI)' 'WTS Porterfield (IRE)'
 'G Gambhir (INDIA)' 'IVA Richards (WI)' 'BRM Tayl

In [30]:
df.head()

,Player,Span,Mat,Inns,NO,Runs,HS,Ave,BF,SR,100,50,0,4s,6s
0,SR Tendulkar (INDIA),1989-2012,463,452,41,18426,200,45,21368,86,49,96,20,2016,195
1,V Kohli (INDIA),2008-2021,254,245,39,12169,183,59,13061,93,43,62,13,1140,125
2,RT Ponting (AUS/ICC),1995-2012,375,365,39,13704,164,42,17046,80,30,82,20,1231,162
3,RG Sharma (INDIA),2007-2021,227,220,32,9205,264,49,10354,89,29,43,13,832,244
4,ST Jayasuriya (Asia/SL),1989-2011,445,433,18,13430,189,32,14725,91,28,68,34,1500,270


NameError: name 'df5' is not defined

In [32]:
selected_players_df = df[df['Player'].isin(['SR Tendulkar (INDIA)', 'V Kohli (INDIA)'])]

In [33]:
df5 = selected_players_df

In [34]:
df5

,Player,Span,Mat,Inns,NO,Runs,HS,Ave,BF,SR,100,50,0,4s,6s
0,SR Tendulkar (INDIA),1989-2012,463,452,41,18426,200,45,21368,86,49,96,20,2016,195
1,V Kohli (INDIA),2008-2021,254,245,39,12169,183,59,13061,93,43,62,13,1140,125


In [38]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Mat,119.0,194.672269,94.879684,33.0,124.5,190.0,247.5,463.0
Inns,119.0,184.411765,89.115276,32.0,120.0,181.0,236.5,452.0
NO,119.0,21.100840,15.818030,1.0,8.0,16.0,33.0,84.0
Runs,119.0,6306.890756,3091.187863,1447.0,3928.5,5964.0,8079.0,18426.0
HS,119.0,151.151261,27.460801,102.0,133.0,145.0,169.5,264.0
Ave,119.0,39.369748,7.015558,24.0,35.0,38.0,42.5,67.0
BF,119.0,7897.941176,3894.847910,1360.0,5086.0,7678.0,9780.5,21368.0
SR,119.0,81.268908,11.366541,56.0,73.5,80.0,88.0,119.0
100,119.0,11.487395,7.577694,5.0,6.0,9.0,14.5,49.0
50,119.0,37.008403,20.005294,4.0,20.5,36.0,47.5,96.0
